In [0]:
%run "/Workspace/Formula1Project/includes/configuration"

In [0]:
%run "/Workspace/Formula1Project/includes/common_functions"

In [0]:
accounts_key = dbutils.secrets.get(scope="formala1-scope", key="account-key")
spark.conf.set("fs.azure.account.key.formala1.dfs.core.windows.net", accounts_key)

In [0]:
dbutils.widgets.text("p_data_source", "2021-03-21")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
qualifying_schema = StructType([ StructField("qualifyId", IntegerType(), False),
                              StructField("raceId", IntegerType(), True), 
                              StructField("driverId", IntegerType(), True), 
                              StructField("ConstructorId", IntegerType(), True), 
                              StructField("number", IntegerType(), True), 
                              StructField("position", StringType(), True),
                              StructField("q1", StringType(), True),
                              StructField("q2", StringType(), True),
                              StructField("q3", StringType(), True),
                              ])

In [0]:
qualifying_df = spark.read.json(f"{raw_path}/{v_file_date}/qualifying",multiLine=True, schema=qualifying_schema)

In [0]:
display(qualifying_df)

In [0]:
from pyspark.sql.functions import current_timestamp, lit
qualifying_df = qualifying_df.withColumnRenamed("qualifyId", "qualify_id").withColumnRenamed("raceId", "race_id").withColumnRenamed("driverId", "driver_id").withColumnRenamed("ConstructorId", "constructor_id").withColumn("ingestion_date",current_timestamp()).withColumn("file_date",lit(v_file_date)).withColumn("data_source",lit(v_data_source))

In [0]:
merge_condition = "tgt.race_id = src.race_id and tgt.qualify_id = src.qualify_id"
merge_data(qualifying_df , processed_path, "f1_processed", "qualifying", "race_id", merge_condition)